In [0]:
import delta

In [0]:
df_full = spark.read.format("parquet").load("/Volumes/raw/sistema_pontos/full_load/clientes/")
(df_full.coalesce(1)
.write
.format("delta")
.mode("overwrite")
.saveAsTable("bronze.sistema_pontos.customers"))

In [0]:
(spark.read
 .format("parquet")
 .load("/Volumes/raw/sistema_pontos/cdc/clientes/")
 .createOrReplaceTempView("customers"))

In [0]:
query = '''
    SELECT *
    FROM customers
    QUALIFY row_number() OVER (PARTITION BY idCliente ORDER BY _cdc_ts DESC) = 1
'''

df_cdc_unique = spark.sql(query)
df_cdc_unique.display()

In [0]:
bronze = delta.DeltaTable.forName(spark, "bronze.sistema_pontos.customers")


In [0]:
(
    bronze.alias("b")
      .merge(df_cdc_unique.alias("d"), "b.idCliente = d.idCliente")
      .whenMatchedDelete(condition = "d.op = 'D'")
      .whenMatchedUpdateAll(condition = "d.op = 'U'")
      .whenNotMatchedInsertAll(condition = "d.op = 'I' or d.op = 'U'")
      .execute()
)


In [0]:
%sql
SELECT *
FROM bronze.sistema_pontos.customers
WHERE idCliente = '0068860b-c7eb-4312-830d-4ab9273c9b15'